Neural Network

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.callbacks import EarlyStopping

Load the dataset and split it into trainings, test and validation data

In [ ]:
# Load the dataset
data = pd.read_pickle('dataset.pkl')

# Extract the validation data from the dataset
val_data = data.loc[data.run >= 97].drop('run', axis=1)
data = data.loc[data.run <= 96].drop('run', axis=1)

# Split the remaining dataset into trainings and test data
train_complete = data[:int(len(data)*0.75)]
test_complete = data[int(len(data)*0.75):]

# Method to drop the less important features (-> Random Forest)
def drop_less_important(data_temp, drop_many=True):
    if drop_many:
        data_temp = data_temp.drop([''], axis=1, errors='ignore')
    return data_temp.drop(['clusterAbsZernikeMoment40', 'clusterAbsZernikeMoment51', 'clusterLAT', 'phi', 'b2bPhi', 'cosAngleBetweenMomentumAndVertexVectorInXYPlane','pRecoilPhi'], axis=1, errors='ignore')

# Split the dataframe containing the pair of photons into two single dataframe containing each on photon and
# one dataframe containing the label
def split_sets(data_temp):
    simple_column_names = ['clusterAbsZernikeMoment40', 'clusterAbsZernikeMoment51', 'clusterErrorTiming', 'clusterLAT', 'clusterTiming', 'clusterZernikeMVA', 'phi', 'pz', 
             'b2bPhi', 'cosAngleBetweenMomentumAndVertexVector', 'cosAngleBetweenMomentumAndVertexVectorInXYPlane', 'pRecoilPhi', 'pxErr', 'pyErr', 'pzErr', 'minC2TDist']
    data_temp_1 = pd.DataFrame(data_temp[[feature + '_1' for feature in simple_column_names]].values, columns=simple_column_names)
    data_temp_2 = pd.DataFrame(data_temp[[feature + '_2' for feature in simple_column_names]].values, columns=simple_column_names)

    return data_temp_1, data_temp_2, data_temp.y

# Method to get data. The trainigs dataset can be reduced by a factor and the less important features can be dropped.
def get_data(factor=1, important_drop=False):

    train_1, train_2, train_y = split_sets(train_complete[:(len(train_complete)//factor)])
    test_1, test_2, test_y = split_sets(test_complete)
    val_1, val_2, val_y = split_sets(val_data)
    
    if important_drop:
        train_1, train_2 = drop_less_important(train_1), drop_less_important(train_2)
        test_1, test_2 = drop_less_important(test_1), drop_less_important(test_2)
        val_1, val_2 = drop_less_important(val_1), drop_less_important(val_2)
    
    print('Length trainings data:', len(train_1), 'Length test data:', len(test_1), 'Length validation data:', len(val_1), '\n')
    return [train_1, train_2, train_y], [test_1, test_2, test_y], [val_1, val_2, val_y]

def get_columns():
    return val_data.columns

data.head(10)

Create the model and train it

In [ ]:
# Get the trainings, test and validation data
factor = 1
train, test, val = get_data(factor=factor, important_drop=True)
print(train[0].columns)

# Define a dropout rate for the 'Dropout' layer
rate = 0.25

# Create the input layers for the two identical neural networks
input_shape_single = len(train[0].columns)
input_shape_l = keras.layers.Input((input_shape_single,))
input_shape_r = keras.layers.Input((input_shape_single,))
output_nodes_last_single_layer = 96


# Create the neural network the first two neural networks share
model_simple = keras.Sequential([
    keras.layers.Input(shape=(input_shape_single,)),
    keras.layers.Dense(128, activation='tanh'),
    #keras.layers.Dropout(rate, noise_shape=None),
    keras.layers.Dense(64, activation='tanh'),
    #keras.layers.Dropout(rate, noise_shape=None),
    keras.layers.Dense(output_nodes_last_single_layer, activation='sigmoid')
])

# Call the the created network on each of the input tensors so the weights will be shared
encoded_l = model_simple(input_shape_l)
encoded_r = model_simple(input_shape_r)
    
# Create a layer to merge the output of the two neural networks together
L1_layer = tf.keras.layers.Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))

# Call this layer with the two previously created networks
L1_distance = L1_layer([encoded_l, encoded_r])

# Create the neural network for the merged values
x = keras.layers.Dense(output_nodes_last_single_layer, activation='tanh')(L1_distance)
#x = keras.layers.Dropout(rate, noise_shape=None)(x)
x = keras.layers.Dense(32, activation='tanh')(x)
#x = keras.layers.Dropout(rate, noise_shape=None)(x)
x = keras.layers.Dense(8, activation='tanh')(x)

# Create a output layer and combine all the models into one
prediction = keras.layers.Dense(1,activation='sigmoid')(x)
siamese_net = keras.models.Model(inputs=[input_shape_l,input_shape_r],outputs=prediction)

# Compile the model
siamese_net.compile(loss="binary_crossentropy",optimizer='Adam', metrics=['accuracy'])

# Set the parameters for the training including early stopping when the validation loss increases (-> overfitting)
num_of_epochs = 350
batch_size = 1024
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=300)

# Train the network
train_history = siamese_net.fit(train[0:2], train[2], verbose=1, batch_size=batch_size, epochs=num_of_epochs, validation_data=[val[0:2], val[2]], callbacks=[es])

Plot the accuracy and loss during training and the best validation accuracy during the training

In [ ]:
def plot_training(loss, acc):
    # Plot the trainings and accuracy history
    
    _, (ax1, ax2) = plt.subplots(2, sharex=True, figsize=(12,8))
    
    ax1.set_ylabel('Loss')
    ax1.plot(loss, '-o')
    
    ax2.set_ylabel('Accuracy')
    ax2.set_xlabel('Epoch Index')
    ax2.plot(acc, '-s')
    
    plt.tight_layout()
    plt.show()

plot_training(train_history.history['loss'], train_history.history['acc'])
plot_training(train_history.history['val_loss'], train_history.history['val_acc'])


print('\nHighest accuracy on the validation dataset during the trainings process:')
print(max(train_history.history['val_acc']))

Evaluate the model: Prints the accuracy and F1-score for the trainings and validation data

In [ ]:
# Print accuracy and F1-score for the test and validation data
y_pred = np.vectorize(round)(siamese_net.predict(test[0:2]))
print('\nAccuracy on test data:', accuracy_score(test[2], y_pred))
print('F1-score on test data:', f1_score(test[2], y_pred))
print('Precision on test data:', precision_score(test[2], y_pred))

y_pred = np.vectorize(round)(siamese_net.predict(val[0:2]))
print('\nAccurcay on validation data:', accuracy_score(val[2], y_pred))
print('F1-score on validation data:', f1_score(val[2], y_pred))
print('Precision on validation data:', precision_score(val[2], y_pred))

Plot the Precision-Recall curve and compute the AUC score

In [ ]:
# Compute the predictions of the classifier as probabilities
y_pred = siamese_net.predict(val[0:2])

# Compute precision, recall and thresholds
precision, recall, thresholds = precision_recall_curve(val[2], y_pred)

# Computes the area under curve from the precision and recall
auc_score = auc(recall, precision)
print('Area under curve:', auc_score)

# Plot the precision-recall curves
plt.plot([0,1], [.33, .33], linestyle='-', label='Random')
plt.axis([0, 1, 0, 1])
plt.plot(recall, precision, linestyle='-', label='Neural Network')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.show()

Save predictions and trainings history

In [ ]:
filename = 'model_9999_data.npy'
data = [siamese_net.predict(test[0:2]), siamese_net.predict(val[0:2]), train_history.history]
np.save(filename, data)